In [26]:
import pandas as pd
import numpy as np
from pulp import *
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence

In [155]:
def GetObjCoeff(m, n, l, X0):
    
    """Get Coefficiencts of Objective Function """
    
    obj_coeff = [1]
    zero_n = list(np.repeat(0, n, axis = 0))
    zero_l = list(np.repeat(0, l, axis = 0))
    input_weighted = (-1/m) * (1/X0)
    
    obj_coeff.extend(zero_n)
    obj_coeff.extend(list(input_weighted))
    obj_coeff.extend(zero_l)
            
    return obj_coeff

def DefineVar(obj_coeff):
    
    """ Define Variables """
    
    n_var = len(obj_coeff)
    variables = LpVariable.dicts("V", list(range(n_var)), lowBound=0, cat="Continuous")
        
    return variables

def SetObjCoeff(obj_coeff, variables, problem):
    
    """ Set Coefficiencts of Objective Function """
    
    obj_func = 0
    for idx, coeff in enumerate(obj_coeff):
        obj_func += coeff * variables[idx]
        
    problem += obj_func
        
    return problem

def LinearConstraint(m, n, l, Y0, variables):
    
    """ Set Linearity condition """
    
    con_coeff = [1]
    zero_n = list(np.repeat(0, n, axis = 0))
    zero_m = list(np.repeat(0, m, axis = 0))
    output_weighted = [(1/l) * (1/Y0)]
    
    con_coeff.extend(zero_n)
    con_coeff.extend(zero_m)
    con_coeff.extend(output_weighted)
    
    constraint = 0
    for idx, coeff in enumerate(con_coeff):
        constraint += coeff * variables[idx]

    return constraint

def InputConstraint(X0, front_X, m, l, variables):
    
    """ Set Input Constraint """

    neg_X0 = -X0
    input_const_list = []
    
    if m == 1:
        diag_mat = [elem for elem_list in -np.identity(m) for elem in list(elem_list)]
        zero_mat = [elem for elem_list in np.zeros((m,l)) for elem in list(elem_list)]
        const_list = []
        const_list.append(float(neg_X0))
        const_list.extend(list(front_X.iloc[0,:]))
        const_list.extend(diag_mat)
        const_list.extend(zero_mat)
        
        constraint = 0
        for idx, coeff in enumerate(const_list):
            constraint += coeff * variables[idx]    
        input_const_list.append(constraint)
        
    else :
        diag_mat = np.identity(m)
        zero_mat = np.zeros((m,l))

        const_mat = pd.concat([neg_X0.reset_index(drop=True),
                               front_X.reset_index(drop=True),
                               pd.DataFrame(diag_mat).reset_index(drop=True), 
                               pd.DataFrame(zero_mat).reset_index(drop=True)], axis=1).to_numpy()

        for each_input in const_mat:
            constraint = 0
            for idx, coeff in enumerate(each_input):
                constraint += coeff * variables[idx]

            input_const_list.append(constraint)

    return input_const_list

def OutputConstraint(Y0, front_Y, m, l, variables):
    
    """ Set Output Constraint for the Current DMU """
        
    neg_Y0 = -Y0
    output_const_list = []
    
    if l == 1:
        diag_mat = [elem for elem_list in -np.identity(l) for elem in list(elem_list)]
        zero_mat = [elem for elem_list in np.zeros((l,m)) for elem in list(elem_list)]
        const_list = []
        const_list.append(float(neg_Y0))
        const_list.extend(list(front_Y.iloc[0,:]))
        const_list.extend(zero_mat)
        const_list.extend(diag_mat)
        
        constraint = 0
        for idx, coeff in enumerate(const_list):
            constraint += coeff * variables[idx]    
        output_const_list.append(constraint)
        
    else :
        diag_mat = np.identity(l)
        zero_mat = zero_mat = np.zeros((l,m))
        const_mat = pd.concat([neg_Y0.reset_index(drop=True),
                               front_Y.reset_index(drop=True),
                               pd.DataFrame(zero_mat).reset_index(drop=True), 
                               pd.DataFrame(diag_mat).reset_index(drop=True)], axis=1).to_numpy() 
        
        for each_output in const_mat:            
            constraint = 0
            for idx, coeff in enumerate(each_output):
                constraint += coeff * variables[idx]
                
            output_const_list.append(constraint)
    
    return output_const_list

def VRSConstraint(n, m, l, variables):
    
    """ Set VRS Constraint for the Current DMU """
    
    lambda_coeff = [-1]
    lambda_coeff_n = list(np.repeat(1, n, axis = 0))
    zero_ms = list(np.repeat(0, m+l, axis = 0))
    
    lambda_coeff.extend(lambda_coeff_n)
    lambda_coeff.extend(zero_ms)
    
    lambda_const = 0
    for idx, coeff in enumerate(lambda_coeff):
        lambda_const += coeff * variables[idx]
        
    return lambda_const

def AllConstWithVRS(problem, X0, Y0, m, n, l, front_X, front_Y, variables):
    
    """ Set VRS Constraints for All DMUs """
    
    linear_coeff = LinearConstraint(m, n, l, Y0, variables)
    problem += linear_coeff == 1.0
    
    input_const = InputConstraint(X0, front_X, m, l, variables)
    for each_input_const in input_const:
        problem += each_input_const == 0
    
    output_const = OutputConstraint(Y0, front_Y, m, l, variables)
    for each_output_const in output_const:
        problem += each_output_const == 0
    
    vrs_const = VRSConstraint(n, m, l, variables)
    problem += vrs_const == 0
    
    return problem

def AllConstWithCRS(problem, X0, Y0, m, n, l, front_X, front_Y, variables):
    
    """ Set CRS Constraints for All DMUs """
    
    linear_coeff = LinearConstraint(m, n, l, Y0, variables)
    problem += linear_coeff == 1.0
    
    input_const = InputConstraint(X0, front_X, m, l, variables)
    for each_input_const in input_const:
        problem += each_input_const == 0
    
    output_const = OutputConstraint(Y0, front_Y, m, l, variables)
    for each_output_const in output_const:
        problem += each_output_const == 0
        
    return problem

def CheckRTS(m, n, l, front_X, front_Y):
    
    """ Check Return to Scale """
    
    eff_list = []
    for idx in list(range(n)):
        X0 = front_X.iloc[:,idx]
        Y0 = front_Y.iloc[:,idx]

        problem = LpProblem("Slack-based Model", LpMinimize)
        obj_coeff = GetObjCoeff(m, n, l, X0)
        variables = DefineVar(obj_coeff)
        problem = SetObjCoeff(obj_coeff, variables, problem)
        problem = AllConstWithVRS(problem, X0, Y0, m, n, l, front_X, front_Y, variables)

        #problem.solve()
        problem.solve(GLPK_CMD(path = './glpk-4.65/w64/glpsol.exe'))
        eff = pulp.value(problem.objective)
        eff_list.append(eff)
    
    eff_one_list = [eff for eff in eff_list if eff >= 1.0]
    if len(eff_one_list) > n/2 :
        return "CRS"
    return "VRS"

def Labeling(n, m, l, id_var, outputs, inputs):
    
    """ Label the Results  """
    
    label_list = [id_var[0], "sbm_vrs_eff"]
    
    for idx in list(range(1,n+1)):
        lambda_label = "lambda_" + str(idx)
        label_list.append(lambda_label)

    input_slack_labels = ['slack_' + each_input for each_input in inputs] 
    label_list.extend(input_slack_labels)
    
    output_slack_labels = ['slack_' + each_output for each_output in outputs] 
    label_list.extend(output_slack_labels)
    
#     for input_idx in list(range(1, m+1)):
#         slack_x_label = "x_slack_" + str(input_idx)
#         label_list.append(slack_x_label)
        
#     for output_idx in list(range(1, l+1)):
#         slack_y_label = "y_slack_" + str(output_idx)
#         label_list.append(slack_y_label)
        
    return label_list

def BenchMarking(n, dmu, df):
    
    """ Add Benchmark Column """
    
    benchmark_list = []
    for row_idx in list(range(0, n)):
        benchmarks = []
        for col_idx, each_lambda in enumerate(list(df.iloc[row_idx,2:n+2])):
            if each_lambda == 0:
                continue
            benchmark = dmu[col_idx]

            if benchmark == dmu[row_idx]:
                continue

            benchmarks.append(str(benchmark))
        benchmark_str = ','.join(benchmarks)
        benchmark_list.append(benchmark_str)
        
    df["benchmark"] = benchmark_list
    
    return df

def DetectOulier(raw_data, n_output):
    
    """ Detect Outlier Using Regresion Residuals """
    
    k = n_output
    s = len(raw_data.columns) - k

    y = raw_data.iloc[:,k]

    outlier_list = []
    for each_x in list(range(k+1, s)):

        x = raw_data.iloc[:,each_x]
        lm = sm.OLS(y, x).fit()
        influence = OLSInfluence(lm)
        sresiduals = influence.resid_studentized_internal
        outlier_list.append(sresiduals.idxmin())
        
    return outlier_list

def AdjustSlacks(df):
    
    final_df = df[df.columns.drop(list(df.filter(regex='lambda')))]
    
    final_df.loc[final_df['slack_foodcost'] < 0, 'slack_foodcost'] = 0
    final_df.loc[final_df['slack_laborcost'] < 0, 'slack_laborcost'] = 0
    final_df.loc[final_df['slack_mktcost'] < 0, 'slack_mktcost'] = 0
    final_df.loc[final_df['slack_othercost'] < 0, 'slack_othercost'] = 0
    final_df.loc[final_df['slack_sales'] < 0, 'slack_sales'] = 0
    
    return final_df    

In [156]:
def DEASolver(raw_data, id_var, outputs, inputs):
    
    """ Main Function """
    
    var_list = id_var + outputs + inputs
    raw_data = raw_data[var_list]
    
    n_output = len(outputs)

    #raw_data = pd.read_csv(input_file_path)
    outlier_list = DetectOulier(raw_data, n_output)
    
    filtered_idx = [i for i in raw_data.index if i not in outlier_list]  
    filtered_data = raw_data.loc[filtered_idx]
    filtered_data = filtered_data.reset_index(drop=True)
    
    input_outputs = filtered_data.iloc[:,1:]
    dmu = filtered_data.iloc[:,0]
    
    l = n_output
    m = len(input_outputs.columns) - l
    n = len(input_outputs)
    front_Y = input_outputs.transpose().iloc[:l,:]
    front_X = input_outputs.transpose().iloc[l:,:]

    RTS = CheckRTS(m, n, l, front_X, front_Y)

    result = []
    for idx in list(range(len(input_outputs))):
        X0 = front_X.iloc[:,idx]
        Y0 = front_Y.iloc[:,idx]

        problem = LpProblem("Slack-based Model", LpMinimize)
        obj_coeff = GetObjCoeff(m, n, l, X0)
        variables = DefineVar(obj_coeff)
        problem = SetObjCoeff(obj_coeff, variables, problem)
        
        if RTS == "VRS":
            problem = AllConstWithVRS(problem, X0, Y0, m, n, l, front_X, front_Y, variables)
        else :
            problem = AllConstWithCRS(problem, X0, Y0, m, n, l, front_X, front_Y, variables)

        problem.solve(GLPK_CMD(path = './glpk-4.65/w64/glpsol.exe'))
        #problem.solve()
        eff = pulp.value(problem.objective)

        solution_list = []
        
        # Do not use 'problem.variable()' ; the order of var is awry  
        for idx, variable in variables.items():

            if idx == 0:
                continue

            if variables[0].varValue == 0:
                each_solution = 0
            else:
                each_solution = variable.varValue/variables[0].varValue
            solution_list.append(each_solution)

        each_result = [eff]
        each_result.extend(solution_list)
        result.append(each_result)

    label_list = Labeling(n, m, l, id_var, outputs, inputs)
    df = pd.concat([dmu.reset_index(drop=True),
                    pd.DataFrame(result).reset_index(drop=True)], axis=1)
    df.columns = label_list

    # Contain lambda columns
    df = BenchMarking(n, dmu, df)
    
    # Drop lambda columns
    #final_df = df[df.columns.drop(list(df.filter(regex='lambda')))]
    final_df = AdjustSlacks(df)    
    
    return final_df

# Data Read

In [104]:
input_file_path = "./data/example_data.csv"
example_df = pd.read_csv(input_file_path)[['ID', 'name', 'year', 'month', 'sales', 'foodcost', 'laborcost', 'mktcost', 'othercost']].dropna()

# 1월

In [159]:
subset = example_df[example_df['month'] == 1]
result = DEASolver(subset, id_var = ['ID'], outputs = ['sales'], inputs = ['foodcost', 'laborcost', 'mktcost', 'othercost'])
result_merged = subset.merge(result, on = 'ID', how = 'left')

C:\Anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## 전체

In [161]:
for each_month in list(range(2,9)):
    
    subset = example_df[example_df['month'] == each_month]
    result = DEASolver(subset, id_var = ['ID'], outputs = ['sales'], inputs = ['foodcost', 'laborcost', 'mktcost', 'othercost'])
    result_merged = pd.concat([result_merged, subset.merge(result, on = 'ID', how = 'left')]).reset_index(drop=True)

In [163]:
result_merged.to_csv("./output/DEA_output_2022_1-9월.csv")